Benötigte Module importieren und Datei laden. Die ersten Zeilen werden ausgegeben.

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import keras_tuner

(train_data, train_col), (test_data, test_col) = tf.keras.datasets.fashion_mnist.load_data()

Ein Bild ausgeben

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

index = 111

plt.imshow(train_data[index], cmap='binary')
plt.xlabel(class_names[train_col[index]])
plt.show()

Definiere Funktion und Tuner-Parameter

In [ ]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    model.add(tf.keras.layers.Dense(
      hp.Choice('units', values=[100, 300, 500]),
      activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = keras_tuner.GridSearch(
    build_model,
    objective='val_accuracy',
    project_name='mnistFashion',
    overwrite=True)

Starte Suchprozedur

In [ ]:
cb_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

tuner.search(train_data, train_col, epochs=100, validation_data=(test_data, test_col), callbacks=[cb_early])

Ermittle die besten Hyperparamter

In [ ]:
best_params_list = tuner.get_best_hyperparameters()
best_params = best_params_list[0]
print(best_params.get('units'))
print(best_params.get('learning_rate'))

Baue das Modell mit den besten Hyperparameter-Einstellungen auf.

In [ ]:
model = tuner.hypermodel.build(best_params)
model.fit(train_data, train_col, epochs=100, validation_data=(test_data, test_col),callbacks=[cb_early])